## Selenium이용한 동적 크롤링 실습 : 주유소 가격 수집

In [1]:
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [4]:
service = Service(executable_path=ChromeDriverManager().install())

## 싼 주유소 찾기 Opinet

- https://www.opinet.co.kr/user/main/mainView.do

### 서울시 구별 주유소 가격 정도 얻어오기

- 사이트에서 [싼주유소찾기] > [지역별] 메뉴 선택
- 지역은 '서울' > 원하는 구 지정
    - '시/군/구'를 선택하는 select option의 XPath 찾기

- '조회'버튼 클릭
- 조회된 주유소 정보 '엑셀저장' 버튼 클릭하여 엑셀파일 다운로드

#### 크롬 드라이버 객체 생성

In [153]:
driver = webdriver.Chrome(service=service)

#### 웹페이지 로드

In [148]:
url = 'https://www.opinet.co.kr/user/main/mainView.do'
driver.get(url)

In [7]:
driver.implicitly_wait(3)

#### [싼주유소찾기]-[지역] 페이지로 이동

- 자바스트립트 코드 실행으로 통해서 이동

In [149]:
driver.execute_script('goSubPage(0,0,99);')

#### 시도 이름 찾기

In [12]:
# 시도 이름목록 select element
xpath = '//*[@id="SIDO_NM0"]'
sidoList = driver.find_element(By.XPATH, xpath)
sidoOpt = sidoList.find_elements(By.TAG_NAME, 'option')
sidoNames = [opt.get_attribute('value') for opt in sidoOpt]
sidoNames

['',
 '서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원특별자치도',
 '충청북도',
 '충청남도',
 '전북특별자치도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

#### 서울시 선택

In [150]:
sido_elem = driver.find_element(By.ID, 'SIDO_NM0')
sido_elem.send_keys('서울특별시')

#### 구 이름 찾기

In [128]:
xpath = '//*[@id="SIGUNGU_NM0"]'
sigunguList = driver.find_element(By.XPATH, xpath)
sigunguOpt = sigunguList.find_elements(By.TAG_NAME, 'option')
sigunguNames = [opt.get_attribute('value') for opt in sigunguOpt]
sigunguNames

['',
 '강남구',
 '강동구',
 '강북구',
 '강서구',
 '관악구',
 '광진구',
 '구로구',
 '금천구',
 '노원구',
 '도봉구',
 '동대문구',
 '동작구',
 '마포구',
 '서대문구',
 '서초구',
 '성동구',
 '성북구',
 '송파구',
 '양천구',
 '영등포구',
 '용산구',
 '은평구',
 '종로구',
 '중구',
 '중랑구']

#### '강남구' 주유소 지정

In [24]:
sigungu_elem = driver.find_element(By.ID, 'SIGUNGU_NM0')
sigungu_elem.send_keys('강남구')

#### '조회' 버튼 클릭

In [26]:
xpath = '//*[@id="searRgSelect"]'
driver.find_element(By.XPATH, xpath).click()

#### 조회된 주유소 '엑셀저장' 버튼 클릭

In [27]:
xpath = '//*[@id="templ_list0"]/div[7]/div/a'
driver.find_element(By.XPATH, xpath).click()

=> '지역_위치별(주유소).xls'파일 다운로드 폴더에 저장됨

### 25개 구 주유소 엑셀파일 다운로드

- 함수로 작성하여 실행

In [79]:
# 함수 임포트
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

In [189]:
driver = webdriver.Chrome(service=service)
url = 'https://www.opinet.co.kr/user/main/mainView.do'
driver.get(url)
driver.implicitly_wait(3)
time.sleep(2)
driver.execute_script('goSubPage(0,0,99);')

In [172]:
 # 서울의 각 구의 싼 주유소 엑셀 저장 함수
def search_download_Gungu(city):
    sido_elem = driver.find_element(By.ID, 'SIDO_NM0')
    sido_elem.send_keys(city)
    time.sleep(2)
    xpath = '//*[@id="SIGUNGU_NM0"]'
    sigunguList = driver.find_element(By.XPATH, xpath)
    sigunguOpt = sigunguList.find_elements(By.TAG_NAME, 'option')
    sigunguNames = [opt.get_attribute('value') for opt in sigunguOpt]

    for n in sigunguNames[1:]:
        sigungu_elem = driver.find_element(By.ID, 'SIGUNGU_NM0')
        sigungu_elem.send_keys(n)
        time.sleep(2)
        xpath = '//*[@id="searRgSelect"]'
        driver.find_element(By.XPATH, xpath).click()
        time.sleep(2)
        xpath = '//*[@id="templ_list0"]/div[7]/div/a'
        driver.find_element(By.XPATH, xpath).click()
        time.sleep(2)

In [190]:
# from tqdm import tqdm
search_download_Gungu('서울특별시')

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=134.0.6998.167)
Stacktrace:
	GetHandleVerifier [0x00A3C7F3+24435]
	(No symbol) [0x009C2074]
	(No symbol) [0x008906E3]
	(No symbol) [0x0087FEC0]
	(No symbol) [0x0089DFDF]
	(No symbol) [0x009046AF]
	(No symbol) [0x0091EB19]
	(No symbol) [0x008FD5B6]
	(No symbol) [0x008CC54F]
	(No symbol) [0x008CD894]
	GetHandleVerifier [0x00D470A3+3213347]
	GetHandleVerifier [0x00D5B0C9+3295305]
	GetHandleVerifier [0x00D5558C+3271948]
	GetHandleVerifier [0x00AD7360+658144]
	(No symbol) [0x009CB27D]
	(No symbol) [0x009C8208]
	(No symbol) [0x009C83A9]
	(No symbol) [0x009BAAC0]
	BaseThreadInitThunk [0x75057BA9+25]
	RtlInitializeExceptionChain [0x76F0C2EB+107]
	RtlClearBits [0x76F0C26F+191]


from tqdm import tqdm
    for n in sigunguNames[1:]:
        sigungu_elem = driver.find_element(By.ID, 'SIGUNGU_NM0')
        sigungu_elem.send_keys(n)
        time.sleep(2)
        xpath = '//*[@id="searRgSelect"]'
        driver.find_element(By.XPATH, xpath).click()
        time.sleep(2)
        xpath = '//*[@id="templ_list0"]/div[7]/div/a'
        driver.find_element(By.XPATH, xpath).click()
        time.sleep(2)

### 엑셀파일 읽어오기

In [198]:
from glob import glob
# glob함수 : 파일경로 및 이름을 모아서 리스트로 저장

In [199]:
import os
path = os.getcwd() # 현재 작업 디렉토리 경로
path

'D:\\workspaces\\04_crawling'

In [200]:
# 엑셀파일 저장 경로
path = 'C:/Users/acorn1/Downloads/'
filename = '지역_위치별*.xls'
filePathList = glob(path + filename)
filePathList

['C:/Users/acorn1/Downloads\\지역_위치별(주유소) (1).xls',
 'C:/Users/acorn1/Downloads\\지역_위치별(주유소) (2).xls',
 'C:/Users/acorn1/Downloads\\지역_위치별(주유소) (3).xls',
 'C:/Users/acorn1/Downloads\\지역_위치별(주유소) (4).xls',
 'C:/Users/acorn1/Downloads\\지역_위치별(주유소) (5).xls',
 'C:/Users/acorn1/Downloads\\지역_위치별(주유소) (6).xls',
 'C:/Users/acorn1/Downloads\\지역_위치별(주유소).xls']

In [181]:
!pip install xlrd

In [201]:
# pandas에서 엑셀 파일 읽어 데이터프레임으로 저장
station = pd.read_excel(filePathList[0], header=2)
station

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
0,서울특별시,구천면주유소,서울 강동구 구천면로 357 (암사동),HD현대오일뱅크,02-441-0536,N,-,1643,1527,-
1,서울특별시,재건에너지 재정제2주유소 고속셀프지점,서울특별시 강동구 천호대로 1246 (둔촌제2동),HD현대오일뱅크,02-487-2030,Y,-,1645,1529,-
2,서울특별시,방아다리주유소,서울 강동구 동남로 811 (명일동),SK에너지,02-442-5145,Y,1855,1655,1535,-
3,서울특별시,대성석유(주)길동주유소,서울 강동구 천호대로 1168,GS칼텍스,02-474-7222,N,1935,1696,1578,1600
4,서울특별시,지에스칼텍스㈜ 동서울주유소,서울 강동구 천호대로 1456 (상일동),GS칼텍스,02-426-5372,Y,1899,1699,1599,-
5,서울특별시,(주)소모 신월주유소,서울 강동구 양재대로 1323,GS칼텍스,02-6956-6674,Y,1916,1716,1598,1500
6,서울특별시,(주)삼표에너지 고덕주유소,서울 강동구 고덕로 39 (암사동),GS칼텍스,02-441-3327,Y,1955,1726,1598,1699
7,서울특별시,sk해뜨는주유소,서울 강동구 상일로 132 (강일동),SK에너지,02-442-7801,Y,1999,1789,1639,1600
8,서울특별시,(주)퍼스트오일 코알라주유소,서울특별시 강동구 올림픽로 556 (성내동),S-OIL,02-484-1162,Y,1970,1790,1670,-
9,서울특별시,주)지유에너지직영 오렌지주유소,서울 강동구 성안로 102 (성내동),SK에너지,02-484-6165,N,1934,1793,1673,-


In [202]:
type(station)

pandas.core.frame.DataFrame

#### 데이터 합치기(세로방향)
- SQL의 UNION과 같은 경우
- 모든 칼럼의 이름이 동일한 경우 상하로 결합
- pandas에서 concat([dfA, dfB, dfC])함수를 이용

In [206]:
dfList = []
for file in filePathList:
    dfList.append(pd.read_excel(file, header=2))
stations = pd.concat(dfList)

In [ ]:
stations.head()

In [207]:
stations.info

<bound method DataFrame.info of        지역                    상호                            주소        상표  \
0   서울특별시                구천면주유소         서울 강동구 구천면로 357 (암사동)  HD현대오일뱅크   
1   서울특별시  재건에너지 재정제2주유소 고속셀프지점  서울특별시 강동구  천호대로 1246 (둔촌제2동)  HD현대오일뱅크   
2   서울특별시               방아다리주유소          서울 강동구 동남로 811 (명일동)     SK에너지   
3   서울특별시          대성석유(주)길동주유소              서울 강동구 천호대로 1168     GS칼텍스   
4   서울특별시        지에스칼텍스㈜ 동서울주유소        서울 강동구 천호대로 1456 (상일동)     GS칼텍스   
..    ...                   ...                           ...       ...   
25  서울특별시         ㈜새서울석유 새서울주유소               서울 강남구 압구정로 154  HD현대오일뱅크   
26  서울특별시          (주)소모 쎈트럴주유소                서울 강남구 삼성로 335     GS칼텍스   
27  서울특별시               SK논현주유소          서울 강남구 논현로 747 (논현동)     SK에너지   
28  서울특별시   (주)제이제이네트웍스 제이제이주유소                서울 강남구 언주로 716  HD현대오일뱅크   
29  서울특별시        (주)만정에너지 삼보주유소         서울 강남구 봉은사로 433 (삼성동)     GS칼텍스   

           전화번호 셀프여부 고급휘발유   휘발유    경유  실내등유  
0   02-441-0536    N

In [208]:
stations.to_csv('crawl_data/서울시주유소.csv', index='False')

-----